In [2]:
import os
from dotenv import load_dotenv

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings.cohere import CohereEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores.elastic_vector_search import ElasticVectorSearch
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate

from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI


# Setup

In [3]:
load_dotenv(".env")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [5]:
with open("./state_of_the_union.txt") as f:
    state_of_the_union = f.read()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_text(state_of_the_union)

embeddings = OpenAIEmbeddings()

docsearch = Chroma.from_texts(
    texts, embeddings, metadatas=[{"source": str(i)} for i in range(len(texts))]
)

AttributeError: 'list' object has no attribute 'split'

In [5]:
query = "What did the president say about Justice Breyer"
docs = docsearch.similarity_search(query)

In [6]:
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.llms import OpenAI

# Shuffling

In [8]:
chain_stuff = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="stuff")
chain_stuff({"input_documents": docs, "question": query}, return_only_outputs=True)

{'output_text': " The president thanked Justice Breyer for his service and mentioned that he nominated Circuit Court of Appeals Judge Ketanji Brown Jackson to continue Justice Breyer's legacy of excellence.\nSOURCES: 31-pl"}

In [12]:
template = """Given the following extracted parts of a long document and a question, create a final answer with references ("SOURCES"). 
If you don't know the answer, just say that you don't know. Don't try to make up an answer.
ALWAYS return a "SOURCES" part in your answer.
Respond in Italian.

QUESTION: {question}
=========
{summaries}
=========
FINAL ANSWER IN ITALIAN:"""
PROMPT = PromptTemplate(template=template, input_variables=["summaries", "question"])

In [13]:
chain_stuff_prompt = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="stuff", prompt=PROMPT)
chain_stuff_prompt({"input_documents": docs, "question": query}, return_only_outputs=True)

{'output_text': '\nIl Presidente non ha fatto alcun riferimento diretto a Justice Breyer nel discorso.\nSOURCES: 31, 21, 34, 10'}

# Map Reduce

In [15]:
chain_map_reduce = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type='map_reduce')
chain_map_reduce({"input_documents": docs, "question": query}, return_only_outputs=True)

{'output_text': ' The president thanked Justice Breyer for his service.\nSOURCES: 31'}

In [16]:
chain_map_reduce_intermediate = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="map_reduce", return_intermediate_steps=True)
chain_map_reduce_intermediate({"input_documents": docs, "question": query}, return_only_outputs=True)

{'intermediate_steps': [' "Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service."',
  ' None',
  ' None',
  ' None'],
 'output_text': ' The president thanked Justice Breyer for his service.\nSOURCES: 31'}

In [17]:
question_prompt_template = """Use the following portion of a long document to see if any of the text is relevant to answer the question. 
Return any relevant text in Italian.
{context}
Question: {question}
Relevant text, if any, in Italian:"""
QUESTION_PROMPT = PromptTemplate(
    template=question_prompt_template, input_variables=["context", "question"]
)

combine_prompt_template = """Given the following extracted parts of a long document and a question, create a final answer with references ("SOURCES"). 
If you don't know the answer, just say that you don't know. Don't try to make up an answer.
ALWAYS return a "SOURCES" part in your answer.
Respond in Italian.

QUESTION: {question}
=========
{summaries}
=========
FINAL ANSWER IN ITALIAN:"""
COMBINE_PROMPT = PromptTemplate(
    template=combine_prompt_template, input_variables=["summaries", "question"]
)

In [19]:
llm = OpenAI(batch_size=5, temperature=0)
chain_map_reduce_intermediate_prompt = load_qa_with_sources_chain(llm, chain_type="map_reduce", return_intermediate_steps=True, question_prompt=QUESTION_PROMPT, combine_prompt=COMBINE_PROMPT)
chain_map_reduce_intermediate_prompt({"input_documents": docs, "question": query}, return_only_outputs=True)

{'intermediate_steps': ["\nStasera vorrei onorare qualcuno che ha dedicato la sua vita a servire questo paese: il giustiziere Stephen Breyer, un veterano dell'esercito, uno studioso costituzionale e un giustiziere in uscita della Corte Suprema degli Stati Uniti. Giustiziere Breyer, grazie per il tuo servizio.",
  ' Non rilevante',
  ' Non rilevante',
  ' Non rilevante'],
 'output_text': "\nIl Presidente ha onorato il Giustiziere Stephen Breyer, veterano dell'esercito, studioso costituzionale e giustiziere in uscita della Corte Suprema degli Stati Uniti, ringraziandolo per il suo servizio.\n\nFONTE: 31"}

# Refine

In [20]:
chain_refine = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="refine")
chain_refine({"input_documents": docs, "question": query}, return_only_outputs=True)

{'output_text': '\n\nThe president said that he wanted to honor Justice Breyer for his dedication to serving the country and his legacy of excellence, while also emphasizing the importance of competition and the need to go after criminals who have stolen relief money meant for small businesses and millions of Americans. He also announced that the Justice Department will name a chief prosecutor for pandemic fraud, and that he was offering a Unity Agenda for the Nation, which included beating the opioid epidemic and investing in America, educating Americans, growing the workforce, and building the economy from the bottom up and the middle out, not from the top down. He also mentioned the importance of passing the Bipartisan Infrastructure Law, the most sweeping investment to rebuild America in history, and that the country would have an infrastructure decade. Source: 21, 34, 10'}

In [22]:
chain_refine_intermediate = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="refine", return_intermediate_steps=True)
chain_refine_intermediate({"input_documents": docs, "question": query}, return_only_outputs=True)

{'intermediate_steps': ['\nThe president said that he wanted to honor Justice Breyer for his dedication to serving the country and his legacy of excellence.',
  '\n\nThe president said that he wanted to honor Justice Breyer for his dedication to serving the country and his legacy of excellence, while also emphasizing the importance of competition and the need to go after criminals who have stolen relief money meant for small businesses and millions of Americans. He also announced that the Justice Department will name a chief prosecutor for pandemic fraud. Source: 21',
  '\n\nThe president said that he wanted to honor Justice Breyer for his dedication to serving the country and his legacy of excellence, while also emphasizing the importance of competition and the need to go after criminals who have stolen relief money meant for small businesses and millions of Americans. He also announced that the Justice Department will name a chief prosecutor for pandemic fraud, and that he was offeri

In [24]:
refine_template = (
    "The original question is as follows: {question}\n"
    "We have provided an existing answer, including sources: {existing_answer}\n"
    "We have the opportunity to refine the existing answer"
    "(only if needed) with some more context below.\n"
    "------------\n"
    "{context_str}\n"
    "------------\n"
    "Given the new context, refine the original answer to better "
    "answer the question (in Italian)"
    "If you do update it, please update the sources as well. "
    "If the context isn't useful, return the original answer."
)
refine_prompt = PromptTemplate(
    input_variables=["question", "existing_answer", "context_str"],
    template=refine_template,
)


question_template = (
    "Context information is below. \n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "Given the context information and not prior knowledge, "
    "answer the question in Italian: {question}\n"
)
question_prompt = PromptTemplate(
    input_variables=["context_str", "question"], template=question_template
)

In [25]:
chain_refine_prompt = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="refine", return_intermediate_steps=True, question_prompt=question_prompt, refine_prompt=refine_prompt)
chain_refine_prompt({"input_documents": docs, "question": query}, return_only_outputs=True)

{'intermediate_steps': ['\nIl presidente ha detto che Justice Breyer ha dedicato la sua vita a servire questo paese e ha reso omaggio a lui per il suo servizio.',
  '\n\nIl presidente ha detto che Justice Breyer ha dedicato la sua vita a servire questo paese e ha reso omaggio a lui per il suo servizio. Ha anche sottolineato che la giustizia è una parte fondamentale della nostra democrazia e che Justice Breyer ha servito con integrità e dedizione.',
  '\n\nIl presidente ha detto che Justice Breyer ha dedicato la sua vita a servire questo paese e ha reso omaggio a lui per il suo servizio. Ha anche sottolineato che la giustizia è una parte fondamentale della nostra democrazia e che Justice Breyer ha servito con integrità e dedizione, contribuendo a rafforzare la legge sulla violenza contro le donne che il presidente aveva scritto tre decenni fa.',
  "\n\nIl presidente ha detto che Justice Breyer ha dedicato la sua vita a servire questo paese e ha reso omaggio a lui per il suo servizio. Ha

# Map-Rerank

In [26]:
chain_map_rerank = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="map_rerank", metadata_keys=['source'], return_intermediate_steps=True)
result = chain_map_rerank({"input_documents": docs, "question": query}, return_only_outputs=True)

In [27]:
result["output_text"]

' The President thanked Justice Breyer for his service and honored him for dedicating his life to serve the country.'

In [30]:
result["source"]

'31'

In [28]:
result["intermediate_steps"]

[{'answer': ' The President thanked Justice Breyer for his service and honored him for dedicating his life to serve the country.',
  'score': '100'},
 {'answer': ' This document does not answer the question', 'score': '0'},
 {'answer': ' This document does not answer the question', 'score': '0'},
 {'answer': ' This document does not answer the question', 'score': '0'}]

In [31]:
from langchain.output_parsers import RegexParser

output_parser = RegexParser(
    regex=r"(.*?)\nScore: (.*)",
    output_keys=["answer", "score"],
)

prompt_template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

In addition to giving an answer, also return a score of how fully it answered the user's question. This should be in the following format:

Question: [question here]
Helpful Answer In Italian: [answer here]
Score: [score between 0 and 100]

Begin!

Context:
---------
{context}
---------
Question: {question}
Helpful Answer In Italian:"""
PROMPT = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"],
    output_parser=output_parser,
)

In [32]:
chain_map_rerank_prompt = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="map_rerank", metadata_keys=['source'], return_intermediate_steps=True, prompt=PROMPT)
query = "What did the president say about Justice Breyer"
result = chain_map_rerank_prompt({"input_documents": docs, "question": query}, return_only_outputs=True)

In [33]:
result

{'source': '31',
 'intermediate_steps': [{'answer': ' Il presidente ha detto che Justice Breyer ha dedicato la sua vita a servire questo paese.',
   'score': '100'},
  {'answer': ' Non so.', 'score': '0'},
  {'answer': ' Non so.', 'score': '0'},
  {'answer': ' Non so.', 'score': '0'}],
 'output_text': ' Il presidente ha detto che Justice Breyer ha dedicato la sua vita a servire questo paese.'}